In [1]:
import tensorflow as tf

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Input, BatchNormalization, ReLU, GlobalAveragePooling2D

import cv2
import numpy as np
import matplotlib.pyplot as plt

import PIL
import os
from os import listdir

2024-07-14 16:07:43.989990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 16:07:43.990066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 16:07:44.021995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 16:07:44.053717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
@keras.saving.register_keras_serializable()
class ResNetBlock(keras.Model):
    def __init__(self, filters, kernel_size = (3,3), strides=1, downsample = False, strides_2 = None):
        super().__init__()
        if strides_2 is None : strides_2 = strides
        self.conv1 = Conv2D(filters, kernel_size, strides = strides, padding = "same")
        self.norm = BatchNormalization()
        self.relu = ReLU()
        self.conv2 = Conv2D(filters, kernel_size, strides = strides_2, padding = "same")
        self.norm2 = BatchNormalization()
        self.downsample = downsample
        if downsample ==True:
            self.downsample_conv = Conv2D(filters, 1, strides)
            self.downsample_norm = BatchNormalization()
        else:
            self.downsample_conv = None
            self.downsample_norm = None
    
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.norm(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu(x)
        y = inputs
        if self.downsample:
            y = self.downsample_conv(y)
            y = self.downsample_norm(y)
        #print(x.shape, y.shape)
        x = x + y
        x = self.relu(x)
        return x

In [10]:
inputs = Input((224, 224,3))
x = Conv2D(64, (7,7), strides = 2, padding="same")(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling2D((2,2))(x)


x = ResNetBlock(64)(x) #Block_1
x = ResNetBlock(64)(x) #Block_2
x = ResNetBlock(64)(x) #Block_3

x = ResNetBlock(128, strides = 2, downsample = True, strides_2 = 1)(x) #Block_4

x = ResNetBlock(128)(x) #Block_5
x = ResNetBlock(128)(x) #Block_6
x = ResNetBlock(128)(x) #Block_7

x = ResNetBlock(256, strides = 2, downsample = True, strides_2 = 1)(x) #Block_4

x = ResNetBlock(256)(x) #Block_9
x = ResNetBlock(256)(x) #Block_10
x = ResNetBlock(256)(x) #Block_11
x = ResNetBlock(256)(x) #Block_12
x = ResNetBlock(256)(x) #Block_13

x = ResNetBlock(512, strides = 2, downsample = True, strides_2 = 1)(x) #Block_14

x = ResNetBlock(512)(x) #Block_15
x = ResNetBlock(512)(x) #Block_16

x = layers.GlobalAveragePooling2D()(x)


outputs = Dense(7, activation = "softmax")(x)

model_resnet = keras.Model(inputs=inputs, outputs=outputs)
model_resnet.summary()

Model: "functional_64"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block (ResNetBlock)     │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_1 (ResNetBlock)   │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_2 (ResNetBlock)   │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_3 (ResNetBlock)   │ (None, 28, 28, 128)    │       231,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_4 (ResNetBlock)   │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_5 (ResNetBlock)   │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_6 (ResNetBlock)   │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_7 (ResNetBlock)   │ (None, 14, 14, 256)    │       921,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_8 (ResNetBlock)   │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_9 (ResNetBlock)   │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_10 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_11 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_12 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_13 (ResNetBlock)  │ (None, 7, 7, 512)      │     3,677,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_14 (ResNetBlock)  │ (None, 7, 7, 512)      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_15 (ResNetBlock)  │ (None, 7, 7, 512)      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,313,799 (81.31 MB)

 Trainable params: 21,296,775 (81.24 MB)

 Non-trainable params: 17,024 (66.50 KB)

In [11]:
data_dir = "Data_Cancer/"
labels = listdir(data_dir)
labels

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [12]:
img_height = 224
img_width = 224
batch_size = 32

In [13]:
def load_images(path, limit=100):
    x = []
    y = []
    labels = os.listdir(path)
    for index, folder in enumerate(os.listdir(path)):
        lim = 0
        for image in os.listdir(path+"/"+folder):
            image_path = path+"/"+folder+"/"+image
            img = cv2.imread(image_path)
            img = np.array(img)
            x.append(img)
            y.append(index)
            lim+=1
            if lim > limit:
                break
    x = np.array(x)
    y = np.array(y)
    return x,y, labels

##### Creating training, validation dataset

In [14]:
x_train, y_train, labels_train = load_images("Data_Cancer/")
x_train = x_train/255.0

In [18]:
x_test, y_test, labels_test = load_images("Data_Cancer_Test/", 50)
x_test = x_test/255.0

In [16]:
epochs = 50
model_resnet.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
history = model_resnet.fit(x_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/50


I0000 00:00:1720966224.357463   10864 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8802', 8 bytes spill stores, 8 bytes spill loads



22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.2343 - loss: 3.3106

I0000 00:00:1720966248.834600   10548 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_39', 160 bytes spill stores, 160 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_146', 48 bytes spill stores, 48 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 39s 618ms/step - accuracy: 0.2360 - loss: 3.2647
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.2692 - loss: 1.7675
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - accuracy: 0.2848 - loss: 1.7719
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.3249 - loss: 1.7099
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.3006 - loss: 1.8374
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.3376 - loss: 1.6233
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.3323 - loss: 1.6426
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.3580 - loss: 1.6291
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - accuracy: 0.4157 - loss: 1.4523
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - accuracy: 0.4326 - loss: 1.4680
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.4322 - loss: 1.4789
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accurac

In [31]:
model_resnet.save("Models/resnet_34_model.keras")

In [8]:
model_vgg = Sequential([
    keras.layers.Input((224,224,3)),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Flatten(),
    
    Dense(4096),
    Dense(4096),
    Dense(1000),
    
    Dense(7, activation = "softmax")
    
])
model_vgg.summary()

2024-07-14 15:12:59.493181: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 15:12:59.689267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 15:12:59.689341: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 15:12:59.691425: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 15:12:59.691483: I external/local_xla/xla/stream_executor

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,00

 Total params: 143,674,247 (548.07 MB)

 Trainable params: 143,674,247 (548.07 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
epochs = 50
model_vgg.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

history_vgg = model_vgg.fit(x_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/50


I0000 00:00:1720962783.810413    6884 service.cc:145] XLA service 0x7f0b540031d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720962783.810468    6884 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-07-14 15:13:03.936028: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-14 15:13:04.404313: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720962785.586007    6922 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 504 bytes spill stores, 460 bytes spill loads

I0000 00:00:1720962785.711556    6921 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 1068 bytes spill stores, 920 bytes spill loads

I0000

22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.1448 - loss: 2.5961

I0000 00:00:1720962838.746972    7063 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 628 bytes spill stores, 580 bytes spill loads

I0000 00:00:1720962838.891904    7060 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 36 bytes spill stores, 36 bytes spill loads

I0000 00:00:1720962839.411781    7055 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1499', 36 bytes spill stores, 36 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 777ms/step - accuracy: 0.1452 - loss: 2.5636
Epoch 2/50


I0000 00:00:1720962847.982853    6884 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_13', 164 bytes spill stores, 164 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_12', 36 bytes spill stores, 36 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 303ms/step - accuracy: 0.1262 - loss: 2.0003
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 291ms/step - accuracy: 0.1457 - loss: 2.0198
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 288ms/step - accuracy: 0.1239 - loss: 1.9800
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 288ms/step - accuracy: 0.1411 - loss: 1.9621
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 288ms/step - accuracy: 0.1430 - loss: 1.9725
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 289ms/step - accuracy: 0.1320 - loss: 1.9652
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 294ms/step - accuracy: 0.1502 - loss: 2.0104
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.1273 - loss: 1.9604
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 295ms/step - accuracy: 0.1596 - loss: 1.9483
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 293ms/step - accuracy: 0.1418 - loss: 1.9648
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.1374 - loss: 1.9631
Epoch 13/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 295ms/step - accurac

In [11]:
model_vgg.save("Models/vgg_19_model.keras")

## Plain 34 Layer Model

In [12]:
model_34 = Sequential([
    Input((img_height, img_width, 3)),
    Conv2D(64, (7,7), strides = 2, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(128, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(256, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(512, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    GlobalAveragePooling2D(),
    Dense(1000),
    Dense(7, activation = "softmax")
    
    
])

model_34.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 14, 14, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 14, 14, 256)    │       590,08

 Total params: 21,623,239 (82.49 MB)

 Trainable params: 21,623,239 (82.49 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
epochs = 50
model_34.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
history_34 = model_34.fit(x_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/50


I0000 00:00:1720963324.678213    8001 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2908', 80 bytes spill stores, 80 bytes spill loads

I0000 00:00:1720963325.396088    8001 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2908', 336 bytes spill stores, 336 bytes spill loads



22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.1137 - loss: 1.9490

I0000 00:00:1720963336.530762    8080 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2908', 100 bytes spill stores, 100 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 25s 390ms/step - accuracy: 0.1147 - loss: 1.9491
Epoch 2/50


I0000 00:00:1720963340.836827    6884 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_32', 36 bytes spill stores, 36 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.1577 - loss: 1.9461
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.1400 - loss: 1.9477
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.1549 - loss: 1.9473
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.1401 - loss: 1.9461
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.1353 - loss: 1.9476
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.1381 - loss: 1.9467
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.1320 - loss: 1.9463
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.1515 - loss: 1.9464
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.1417 - loss: 1.9471
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.1343 - loss: 1.9470
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.1379 - loss: 1.9472
Epoch 13/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.1682 - 

In [14]:
model_34.save("Models/plain_34_model.keras")

## Predictions for all the models

In [ ]:
model_vgg = keras.saving.load_model("Models/vgg_19_model.keras")
predictions = model_vgg.predict(x_test)
predictions = np.argmax(predictions, axis = 1)

print(tf.math.confusion_matrix(predictions = predictions, labels = y_test))
count = 0
tp = 0
for pred in predictions:
    if pred == y_test[count]:
        tp +=1
    count+=1
    
print("Accuracy for the test : ", tp/count)

In [21]:
model_34 = keras.saving.load_model("Models/plain_34_model.keras")

predictions = model_34.predict(x_test)
predictions = np.argmax(predictions, axis = 1)
print(tf.math.confusion_matrix(predictions = predictions, labels = y_test))

count = 0
tp = 0

for pred in predictions:
    if pred == y_test[count]:
        tp +=1
    count+=1
    
print("Accuracy for the test : ", tp/count)

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step
tf.Tensor(
[[ 0  0  0  0  0 43  0]
 [ 0  0  0  0  0 51  0]
 [ 0  0  0  0  0 51  0]
 [ 0  0  0  0  0 44  0]
 [ 0  0  0  0  0 51  0]
 [ 0  0  0  0  0 51  0]
 [ 0  0  0  0  0 35  0]], shape=(7, 7), dtype=int32)
Accuracy for the test :  0.15644171779141106


In [20]:
#model_resnet = keras.saving.load_model("Models/resnet_34_model.keras")

predictions = model_resnet.predict(x_test)
predictions = np.argmax(predictions, axis = 1)

print(tf.math.confusion_matrix(predictions = predictions, labels = y_test))

count = 0
tp = 0

for pred in predictions:
    if pred == y_test[count]:
        tp +=1
    count+=1
    
print("Accuracy for the test : ", tp/count)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
tf.Tensor(
[[ 3  0  5 25  9  1  0]
 [ 1 11  8 23  5  3  0]
 [ 1  3 17 10 16  4  0]
 [ 1  5  3 25  8  2  0]
 [ 0  3  7  1 40  0  0]
 [ 2  1  7  7 29  5  0]
 [ 1 12  2 12  4  1  3]], shape=(7, 7), dtype=int32)
Accuracy for the test :  0.31901840490797545


In [19]:
from collections import Counter
dc_labels = Counter(y_test)
for i in range(7):
    print(labels_test[i] ,i ,dc_labels[i])

akiec 0 43
bcc 1 51
bkl 2 51
df 3 44
mel 4 51
nv 5 51
vasc 6 35
